<a href="https://colab.research.google.com/github/Bharath-SN/Twitter_Sentiment_analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing kaggle library
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.37GB/s]


In [ ]:
from zipfile import ZipFile
file_name = "/content/sentiment140.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done extraction')

done extraction


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
#loading the dataset
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [ ]:
#check the number of rows and columns
twitter_data.shape

(1599999, 6)

In [ ]:
#printing first 5 rows
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#since column names are not shown above we are adding column names
column_nanes = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv',names=column_nanes, encoding = 'ISO-8859-1')

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#Checking null values
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
#Checking the dstribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
#convert the target "4" to "1" so if 1 means Positive
twitter_data.replace({'target':{4:1}}, inplace=True)


In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [ ]:
port_stem = PorterStemmer()

In [ ]:
def  stemming(content):
    stemmed_content = re.sub('[^a-zA-z)]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.to_csv("training.1600000.processed.noemoticon.csv", index=False)

In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data into training and testing

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X , Y , test_size=0.2 , stratify = Y, random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [ ]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [ ]:
#converting the textuall data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9418318 stored elements and shape (1280000, 486129)>
  Coords	Values
  (0, 460631)	0.2722796751905946
  (0, 374738)	0.3583777141676211
  (0, 195880)	0.5274003168649051
  (0, 116938)	0.37495943913789
  (0, 248849)	0.42150112510914717
  (0, 467183)	0.44835289535143485
  (1, 170368)	1.0
  (2, 116938)	0.45907357295517354
  (2, 132860)	0.189185243396702
  (2, 430009)	0.18705550082972291
  (2, 138040)	0.29075008205447994
  (2, 429073)	0.32103263007778204
  (2, 457418)	0.32959283442456544
  (2, 83924)	0.31302589595853153
  (2, 467557)	0.33479215858146927
  (2, 282477)	0.24136142115953255
  (2, 431956)	0.15168315144179975
  (2, 188469)	0.16193348841898428
  (2, 160107)	0.18800042842745246
  (2, 141023)	0.20289102989813276
  (2, 305576)	0.16785750145215705
  (3, 429073)	0.2902621563752629
  (3, 168339)	0.44575321204437185
  (3, 161199)	0.27849689984807324
  (3, 61325)	0.5200757108578115
  :	:
  (1279996, 336672)	0.2119062618474879
  

In [ ]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2275698 stored elements and shape (320000, 486129)>
  Coords	Values
  (0, 18045)	0.17179904535444512
  (0, 35034)	0.16235855114931586
  (0, 73304)	0.26783408223486305
  (0, 113521)	0.36706198374481314
  (0, 141078)	0.25508294654706937
  (0, 147070)	0.23682500848978227
  (0, 181513)	0.28051909372627465
  (0, 287140)	0.4532068414387133
  (0, 295762)	0.1781246309025408
  (0, 409981)	0.2197625898683916
  (0, 421087)	0.3488277632911353
  (0, 431956)	0.3140992368895445
  (0, 444364)	0.17902870421365852
  (1, 8846)	0.30376212070973824
  (1, 18045)	0.2084951459364015
  (1, 154565)	0.5722647009624491
  (1, 230517)	0.40557675638155133
  (1, 271981)	0.28432510390186794
  (1, 367832)	0.48204720620439945
  (1, 386889)	0.24304049279900844
  (2, 25943)	0.35298537119626666
  (2, 38352)	0.3793962504534252
  (2, 96014)	0.3631508764925503
  (2, 193932)	0.5885511503893769
  (2, 272055)	0.2563052640008033
  :	:
  (319994, 467935)	0.2780055449294

Training the machine learning model
Logistic regression -
classification

In [ ]:
model = LogisticRegression(max_iter=1000)


In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**
Accuracy score

In [ ]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [ ]:
print('accuracy score of the training data:',training_data_accuracy)

accuracy score of the training data: 0.79172421875


In [ ]:
#accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)

In [ ]:
print('accuracy score of the training data:',test_data_accuracy)

accuracy score of the training data: 0.776003125


saving the model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

using the saved model for future predictions

In [ ]:
#loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative tweet')
else:
  print('Positive tweet')

1
[1]
Positive tweet


In [ ]:
X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative tweet')
else:
  print('Positive tweet')

0
[0]
Negative tweet
